In [ ]:
from os.path import join

def setup_file_system(in_colab):
    if in_colab:
        from google.colab import drive

        # Set the base and mount path
        MOUNT_PATH_DRIVE = '/content/drive'
        BASE_PATH = join(
            MOUNT_PATH_DRIVE, 
            "MyDrive/project_asr"
        )

        # Mount the google drive
        drive.mount(MOUNT_PATH_DRIVE)

        return BASE_PATH

    else:
        return "/workspaces/project_automated_sound_recognition"

In [ ]:
import sys
from os import chdir
from os.path import join

# Method to check if the notebook is running in colab or local
IN_COLAB = 'google.colab' in sys.modules

# Set the base path of the project
BASE_PATH = setup_file_system(IN_COLAB)

# Set the base path of the project
chdir(join(BASE_PATH, "src/"))

In [ ]:
%load_ext autoreload
%autoreload 2

# Imports
# Utils
import matplotlib as plt
import numpy as np
import wandb
import sys
import importlib
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import datetime
import json
from sklearn.metrics import accuracy_score, confusion_matrix


# DL libraries
import torch
import torch.optim as optim
from torch import nn
import torch.utils.data 
from torch.utils.data import DataLoader

# User libraries
from dataset.audio_sample_dataset import AudioSampleDataset
from model.baseline_model import BaselineModel
from trainer.trainer import train_classification_model
from validator.validator import validate_classification_model
from util import config, util_functions, model_management

In [ ]:
def set_augmentations(wandb_config):
    train_augmentations = {
        'pitch_shift': {
            'enabled': False,
        },
        'noise': {
            'enabled': True,
            'p': wandb_config.p_noise,
            'min_amplitude': 0.001,
            'max_amplitude': 0.015,
        },
        'mixup': {
            'enabled': True,
            'p': wandb_config.p_mixup,
            'alpha': 0.2,
        },
        'freq_mask': {
            'enabled': True,
            'p': wandb_config.p_freq_mask,
            'freq_mask_param': 5,
        },
        'time_mask': {
            'enabled': True,
            'p': wandb_config.p_time_mask,
            'time_mask_param': 10,
        }
    }

    test_augmentations = {
        'pitch_shift': {
            'enabled': False,
        },
        'noise': {
            'enabled': False,
        },
        'mixup': {
            'enabled': False,
        },
        'freq_mask': {
            'enabled': False,
        },
        'time_mask': {
            'enabled': False,
        }
    }

    return train_augmentations, test_augmentations

In [ ]:
def get_dataloaders(train_augmentations, test_augmentations):
    # Get the train and test data
    train_dataset = AudioSampleDataset(
            join(BASE_PATH, config.TRAIN_DATA_PATH),
            train_augmentations
        )
    test_dataset = AudioSampleDataset(
            join(BASE_PATH, config.TEST_DATA_PATH),
            test_augmentations
        )

    # Place in dataloaders
    train_dataloader = DataLoader(train_dataset, batch_size=config.BATCH_SIZE, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=1)

    return train_dataloader, test_dataloader

In [ ]:
def train():
    # Initialize a new wandb run
    with wandb.init():
        # this config will be set by Sweep Controller
        wandb_config = wandb.config

        # Set the augmentations
        train_augmentations, test_augmentations = set_augmentations(wandb_config)

        # Set the dataloaders
        train_dataloader, test_dataloader = get_dataloaders(train_augmentations, test_augmentations)
        
        # Clear gpu cache
        torch.cuda.empty_cache()

        # Get the model
        model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
        model.fc = nn.Sequential(
            nn.Linear(in_features=512, out_features= 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(in_features=256, out_features=len(config.LABELS)),
            nn.Softmax(dim= 1)
        )
        model.to(config.DEVICE)

        # Set the optimizer
        optimizer = optim.Adam(model.parameters(), lr=config.LR)

        # Set the loss fn
        criteria = nn.CrossEntropyLoss()

        # Set the gradient scaler
        grad_scaler = torch.cuda.amp.grad_scaler.GradScaler()

        # Start training
        for epoch in range(wandb_config.epochs):
            # Set the model in training mode
            model.train()
            
            # Train the model
            total_train_loss_this_epoch = train_classification_model(
                model,
                optimizer,
                criteria,
                grad_scaler,
                train_dataloader
            )
            
            # Set the model in evaluation mode
            model.eval()

            # Validate the model
            total_val_loss_this_epoch, pred_classes, true_classes = validate_classification_model(
                model,
                criteria,
                test_dataloader,
            )

            # Calculate the loss values
            train_loss_this_epoch = total_train_loss_this_epoch/len(train_dataloader.dataset)
            val_loss_this_epoch = total_val_loss_this_epoch/len(test_dataloader.dataset)

            # Calculate the accuracy
            acc_avg = accuracy_score(true_classes, pred_classes)

            # Calculate acc per class
            matrix = confusion_matrix(true_classes, pred_classes)
            acc_per_class = matrix.diagonal()/matrix.sum(axis=1)


            # Log the train loss this epoch
            wandb.log({
                'train_loss': train_loss_this_epoch,
                'val_loss': val_loss_this_epoch,
                'acc': acc_avg,
                'acc_airport': acc_per_class[0],
                'acc_shopping_mall': acc_per_class[1],
                'acc_metro_station': acc_per_class[3],
                'acc_street_pedestrian': acc_per_class[3],
                'acc_public_square': acc_per_class[4],
                'acc_street_traffic': acc_per_class[5],
                'acc_tram': acc_per_class[6],
                'acc_bus': acc_per_class[7],
                'acc_metro': acc_per_class[8],
                'acc_park': acc_per_class[9],
            })

            print(f'epoch: {epoch}, train_loss: {train_loss_this_epoch}, val_loss: {val_loss_this_epoch}, acc: {acc_avg}') 



In [ ]:
# Setup weights and biasses
wandb.login()

# Set sweep config
sweep_config = {
    'method': 'bayes',
    'metric': {
        'name': 'acc',
        'goal': 'maximize'   
    },
    'parameters': {
        'p_noise': {
            'values': [0.0, 0.25, 0.5, 0.75, 1.0]
        },
        'p_mixup': {
            'values': [0.0, 0.25, 0.5, 0.75, 1.0]
        },
        'p_freq_mask': {
            'values': [0.0, 0.25, 0.5, 0.75, 1.0]
        },
        'p_time_mask': {
            'values': [0.0, 0.25, 0.5, 0.75, 1.0]
        },
        'epochs': {
            'value': 50
        },
    }
}

# Send a request to wandb servers, telling that you want to start a sweep
# If not initialized, create with "wandb.sweep(sweep_config, project="project_asr")"
sweep_id = 'mamtu4dy'

# Start an agent that runs train a number of times with a randomly generated config
wandb.agent(sweep_id= sweep_id, function= train, project= 'project_asr')